In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms

from tqdm import tqdm

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available()
                      else "cuda" if torch.cuda.is_available()
                      else "cpu")
device

device(type='mps')

In [3]:
alexnet = models.alexnet(weights=models.AlexNet_Weights.DEFAULT).to(device)

optimizer = optim.Adam(alexnet.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss()

In [4]:
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
alexnet.classifier = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(in_features=9216, out_features=4096),
    nn.ReLU(True),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(True),
    nn.Linear(4096, 10)
)

In [5]:
train_transform = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),

    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2470, 0.2435, 0.2616))
])

In [6]:
train = datasets.CIFAR10("../../data", train=True, transform=train_transform,
                         download=True)
test = datasets.CIFAR10("../../data", train=False, transform=test_transform,
                        download=True)

train_dataloader = DataLoader(train, batch_size=2048, shuffle=True,
                              generator=torch.Generator().manual_seed(42))

test_dataloader = DataLoader(test, batch_size=2048, shuffle=False,
                             generator=torch.Generator().manual_seed(42))

In [8]:
num_epochs = 10

alexnet.train()
for epoch in range(1, num_epochs + 1):
    running_loss = 0.0
    for imgs, labels in tqdm(train_dataloader):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        preds = alexnet(imgs)
        loss = criterion(preds, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"{epoch}/{num_epochs} Loss: {running_loss}")

  0%|          | 0/25 [00:00<?, ?it/s]


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small

In [ ]:
alexnet.eval()
with torch.no_grad():
    output = alexnet(batch_t)

In [ ]:
proba = output[0]
pred = torch.argmax(output[0]).item()
print(category[pred])
print("accuracy: ", proba[pred].item())


diamondback
accuracy:  8.328636169433594
